# Train Domain Adaptation Models

### Make necessary imports

In [ ]:
import torch

from train_cycle_gan import train_cycle_gan
from train_dual_gans import train_dual_gans

### Setup few parameters

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
models = {'dualGAN', 'cycleGAN'}
data_root = "./gta/images/"

In [ ]:
def train_model(model, semi_supervised=False):
    if model in models:
        if model == 'dualGAN':
            train_dual_gans(data_root, semi_supervised)
        else:
            train_cycle_gan(data_root, semi_supervised)

### Specify the model and whether semi-supervised is required or not and then call the train_model function

In [ ]:
model = 'dualGAN'
semisupervised = False
train_model(model, semisupervised)